In [10]:
import numpy as np
import networkx as nx
import random
import matplotlib.pyplot as plt
from matplotlib import animation

In [ ]:
with open(GNN_data_dict, "rb") as file:
    GNN_data_d = pickle.load(file)

GNN_data_list = []
# 遍历字典中的每个键值对
for key, value_list in GNN_data_d.items():
    # 将当前键的列表扩展到合并后的列表中
    GNN_data_list.extend(value_list)
# 打印合并后的列表

train_data, test_data = train_test_split(GNN_data_list, test_size=0.1)
len(train_data), len(test_data),

(534, 60)

In [ ]:
test_data[0]

Data(pos=[114, 3], cell=[1, 3, 3], atomic_numbers=[114], natoms=114, tags=[114], edge_index=[2, 5547], cell_offsets=[5547, 3], force=[114, 3], fixed=[114], y_init=-0.19955779999996714, y_relaxed=-0.3541478000000087, pos_relaxed=[114, 3])

In [ ]:
a2g_test = AtomsToGraphs(
    max_neigh=12,  # 设置每个原子最多与多少个相邻的原子建立联系。
    radius=2,  # 设置建立相邻原子之间边的最大距离。在此半径内的原子将被视为相邻原子。
    r_energy=True,  # False for test data 设置是否将能量作为节点特征添加到图中，默认为True，表示将能量添加到图中。
    r_forces=True,  # 设置是否将力作为节点特征添加到图中，默认为True，表示将力添加到图中。
    r_distances=False,  # 设置是否将原子之间的距离作为特征添加到图中，默认为False，表示不添加距离特征。
    r_fixed=True,  # 设置是否将固定性作为节点特征添加到图中，默认为True，表示将固定性添加到图中
)

In [ ]:
import networkx as nx
import torch_geometric

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors as m2colors


mcolors = dict(m2colors.BASE_COLORS, **m2colors.CSS4_COLORS)


def plot_atom_graph(data_obj):
    """
    visualize atomic graph (useful for verifying type / number of elements, size of graph)
    """

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    node_labels = [str(int(i)) for i in data_obj.atomic_numbers.numpy()]
    # print(node_labels)
    g = torch_geometric.utils.to_networkx(
        data_obj, to_undirected=True, node_attrs=["atomic_numbers"]
    )

    carac = pd.DataFrame(
        {
            "ID": np.arange(len(node_labels)),
            "type": data_obj.atomic_numbers.numpy(),
            "size": data_obj.atomic_numbers.numpy(),
        }
    )  # / np.amax(data_obj.atomic_numbers.numpy())

    carac = carac.set_index("ID")
    carac = carac.reindex(g.nodes())
    carac["type"] = pd.Categorical(carac["type"])
    
    color_list = [
    (1.0, 0.0, 0.0),  # 红色
    (0.0, 1.0, 0.0),  # 绿色
    (0.0, 0.0, 1.0),  # 蓝色
    (0.5, 0.0, 0.5),  # 紫色
    (1.0, 1.0, 0.0),  # 黄色
    (0.5, 0.5, 0.5),  # 灰色
    (1.0, 0.647, 0.0) # 橙色
    ]
    
    # 使用RGB颜色列表创建ListedColormap对象
    cmap = matplotlib.colors.ListedColormap(color_list)

    G=g
    # for v in G:
    #     print(v)
    #     print(pos[v])
    pos = nx.spectral_layout(G, dim=3)
    nodes = np.array([pos[v] for v in G])
    print(nodes)
    edges = np.array([(pos[u], pos[v]) for u, v in G.edges()])

    # Draw graph
    # 绘制节点
    
    ax.scatter(*nodes.T, alpha=0.2, s=100, color="blue")

    # 绘制边
    for edge in edges:
        ax.plot(*edge.T, color="gray")
    # can use edgecolors attribute to color border of nodes by tag

    return ax

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import torch_geometric
import matplotlib.pyplot as plt
from matplotlib import colors as m2colors

# 假设其他必要的库（如 torch_geometric, pandas, numpy 等）已经被正确导入

def plot_atom_graph(data_obj, spectral=False):
    """
    visualize atomic graph (useful for verifying type / number of elements, size of graph)
    """

    # data_obj.atomic_numbers = data_obj.atomic_numbers +data_obj.fixed
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    g = torch_geometric.utils.to_networkx(
        data_obj, to_undirected=True, node_attrs=["atomic_numbers"]
    )

    carac = pd.DataFrame(
        {
            "ID": np.arange(g.number_of_nodes()),
            "type":  (data_obj.atomic_numbers+data_obj.fixed).numpy(),
            "size": data_obj.atomic_numbers.numpy(),
        }
    )

    carac = carac.set_index("ID")
    carac = carac.reindex(g.nodes())
    carac["type"] = pd.Categorical(carac["type"])
    
    color_list = [
        (1.0, 0.0, 0.0),  # 红色
        (170/255, 62/255, 0.0),  # 绿色
        (200/255, 128/255, 51/255), # 橙色
        (0.0, 0.0, 1.0),  # 蓝色
        (0.5, 0.0, 0.5),  # 紫色
        (1.0, 1.0, 0.0),  # 黄色
        (0.5, 0.5, 0.5),  # 灰色
        
    ]
    
    # 使用RGB颜色列表创建ListedColormap对象
    cmap = matplotlib.colors.ListedColormap(color_list)
    
    pos = data_obj.pos.numpy()
    r = range(len(pos))
    pos = dict(zip(r, pos))
    if spectral:
        pos = nx.spectral_layout(g, dim=3)  # 生成三维位置
        print(pos)
    print(pos)
    nodes = np.array([pos[v] for v in sorted(g.nodes())])

    # 绘制节点，使用原子类型的分类代码作为索引来选择颜色
    node_colors = [cmap.colors[code] for code in carac['type'].cat.codes]
    ax.scatter(*nodes.T, s=carac['size']*2+40, c=node_colors, alpha=0.7,edgecolors='black')

    # 绘制边
    edges = np.array([(pos[u], pos[v]) for u, v in g.edges()])
    for edge in edges:
        ax.plot(*edge.T, color="grey", linewidth= 0.4 )

    # ax.grid(False)
    # ax.set_axis_off()
    # plt.tight_layout()
    cube_definition = np.concatenate([np.array([[.0,.0,.0]]),data_obj.cell.numpy()[0]],axis=0)
    cube_definition_array = [
    np.array(list(item))
    for item in cube_definition
    ]

    points = []
    points += cube_definition_array
    vectors = [
        cube_definition_array[1] - cube_definition_array[0],
        cube_definition_array[2] - cube_definition_array[0],
        cube_definition_array[3] - cube_definition_array[0]
    ]

    points += [cube_definition_array[0] + vectors[0] + vectors[1]]
    points += [cube_definition_array[0] + vectors[0] + vectors[2]]
    points += [cube_definition_array[0] + vectors[1] + vectors[2]]
    points += [cube_definition_array[0] + vectors[0] + vectors[1] + vectors[2]]

    points = np.array(points)

    cubeedges = [
        [points[0], points[1]],
        [points[0], points[2]],
        [points[0], points[3]],
        [points[1], points[4]],
        [points[1], points[5]],
        [points[2], points[4]],
        [points[2], points[6]],
        [points[3], points[5]],
        [points[3], points[6]],
        [points[4], points[7]],
        [points[5], points[7]],
        [points[6], points[7]]
    ]
    if not spectral:
        # Plot the edges
        for edge in cubeedges:
            ax.plot([edge[0][0], edge[1][0]], [edge[0][1], edge[1][1]], [edge[0][2], edge[1][2]], 'k')

    # Hide the axes
    ax.set_axis_off()

    # Set equal scaling
    ax.set_box_aspect([np.ptp(a) for a in zip(*points)])  # 'ptp' stands for 'peak to peak'

    plt.show()
    

    return ax


In [ ]:
%matplotlib auto

plot_atom_graph(test_data[0])

Using matplotlib backend: QtAgg
{0: array([1.2937, 0.739 , 0.    ], dtype=float32), 1: array([2.5874, 1.4781, 2.1021], dtype=float32), 2: array([0.    , 0.    , 4.2043], dtype=float32), 3: array([1.2937, 0.739 , 6.3064], dtype=float32), 4: array([2.5874, 1.4781, 8.4086], dtype=float32), 5: array([ 0.    ,  0.    , 10.5069], dtype=float32), 6: array([ 1.2937,  0.739 , 12.584 ], dtype=float32), 7: array([2.5739, 2.9558, 0.    ], dtype=float32), 8: array([3.8676, 3.6949, 2.1021], dtype=float32), 9: array([1.2802, 2.2168, 4.2043], dtype=float32), 10: array([2.5739, 2.9558, 6.3064], dtype=float32), 11: array([3.8676, 3.6949, 8.4086], dtype=float32), 12: array([ 1.2802,  2.2168, 10.5069], dtype=float32), 13: array([ 2.5739,  2.9558, 12.584 ], dtype=float32), 14: array([3.8541, 5.1726, 0.    ], dtype=float32), 15: array([5.1477, 5.9117, 2.1021], dtype=float32), 16: array([2.5604, 4.4336, 4.2043], dtype=float32), 17: array([3.8541, 5.1726, 6.3064], dtype=float32), 18: array([5.1477, 5.9117, 8.

<Axes3D: >

In [ ]:
plot_atom_graph(test_data[0],spectral=True)

{0: array([ 0.2256,  0.3239, -0.3067]), 1: array([ 0.1774,  0.063 , -0.0013]), 2: array([ 0.1201, -0.052 ,  0.0202]), 3: array([ 0.0082, -0.2072,  0.0428]), 4: array([-0.1076, -0.0907, -0.0795]), 5: array([-0.1677, -0.0227, -0.152 ]), 6: array([-0.2175,  0.206 , -0.1258]), 7: array([ 0.2219,  0.2896, -0.2196]), 8: array([0.1771, 0.0599, 0.0027]), 9: array([ 0.1199, -0.0513,  0.0206]), 10: array([ 0.0081, -0.2053,  0.0494]), 11: array([-0.1058, -0.0972, -0.0692]), 12: array([-0.1717,  0.0203, -0.0256]), 13: array([-0.2193,  0.2217, -0.1837]), 14: array([ 0.2181,  0.2575, -0.1612]), 15: array([0.1772, 0.0598, 0.0025]), 16: array([ 0.1177, -0.0661,  0.0366]), 17: array([ 0.0081, -0.2083,  0.0514]), 18: array([-0.1029, -0.1154, -0.0684]), 19: array([-0.1718,  0.0209, -0.0278]), 20: array([-0.2153,  0.1932, -0.1613]), 21: array([ 0.2181,  0.2577, -0.1634]), 22: array([0.1774, 0.0623, 0.    ]), 23: array([ 0.1157, -0.0786,  0.0489]), 24: array([ 0.0083, -0.2103,  0.0449]), 25: array([-0.103 

<Axes3D: >

In [11]:
G = nx.dodecahedral_graph()
pos = nx.spectral_layout(G, dim=3)
nodes = np.array([pos[v] for v in G])
edges = np.array([(pos[u], pos[v]) for u, v in G.edges()])

In [13]:
%matplotlib auto

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# 假设 nodes 和 edges 已经被定义为包含坐标信息的数组
# nodes 应该是一个形状为 (N, 3) 的数组，其中 N 是点的数量
# edges 是一个列表，其中每个元素都是一个形状为 (2, 3) 的数组，表示一条边的两个端点

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# 绘制节点
ax.scatter(*nodes.T, alpha=0.2, s=100, color="blue")

# 绘制边
for edge in edges:
    ax.plot(*edge.T, color="gray")

ax.grid(False)  # 关闭网格
ax.set_axis_off()  # 关闭坐标轴显示
plt.tight_layout()  # 调整布局
plt.show()  # 显示图形


Using matplotlib backend: QtAgg
